<a href="https://colab.research.google.com/github/AkhilVinayakp/HAR_LSTM_CNN/blob/main/CNN_Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My\ Drive/datasets 
%pwd

/content/drive/My Drive/datasets


'/content/drive/My Drive/datasets'

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [8]:

df =  pd.read_csv('inter_norm_mini.csv')

In [4]:
df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,14,Downstairs,57725432324000,17.97,19.500000,-4.400000
1,7,Upstairs,208866321473000,2.91,5.600000,-1.040000
2,29,Jogging,0,2.82,8.710000,-3.541290
3,15,Jogging,0,10.69,3.316667,-1.698003
4,8,Jogging,111020251899000,-1.54,-8.730000,-8.620000


In [5]:
df

,user,activity,timestamp,x-axis,y-axis,z-axis
0,14,Downstairs,57725432324000,17.97,19.500000,-4.400000
1,7,Upstairs,208866321473000,2.91,5.600000,-1.040000
2,29,Jogging,0,2.82,8.710000,-3.541290
3,15,Jogging,0,10.69,3.316667,-1.698003
4,8,Jogging,111020251899000,-1.54,-8.730000,-8.620000
...,...,...,...,...,...,...
105,23,Downstairs,1749732310000,4.79,9.530000,-0.110000
106,31,Downstairs,24512351662000,-1.38,8.120000,2.910000
107,14,Jogging,59899442333000,11.60,12.180000,-1.460000
108,19,Walking,129874801576000,14.10,5.670000,-0.230000


In [9]:
#droping the timestamp and user column for testing
df.drop('user', axis = 1, inplace=True)
df.drop('timestamp', axis = 1, inplace=True)

In [10]:
df

,activity,x-axis,y-axis,z-axis
0,Downstairs,17.97,19.500000,-4.400000
1,Upstairs,2.91,5.600000,-1.040000
2,Jogging,2.82,8.710000,-3.541290
3,Jogging,10.69,3.316667,-1.698003
4,Jogging,-1.54,-8.730000,-8.620000
...,...,...,...,...
105,Downstairs,4.79,9.530000,-0.110000
106,Downstairs,-1.38,8.120000,2.910000
107,Jogging,11.60,12.180000,-1.460000
108,Walking,14.10,5.670000,-0.230000


In [13]:
le = LabelEncoder()
df['activity'] = le.fit_transform(df['activity'])

In [14]:
df

,activity,x-axis,y-axis,z-axis
0,0,17.97,19.500000,-4.400000
1,4,2.91,5.600000,-1.040000
2,1,2.82,8.710000,-3.541290
3,1,10.69,3.316667,-1.698003
4,1,-1.54,-8.730000,-8.620000
...,...,...,...,...
105,0,4.79,9.530000,-0.110000
106,0,-1.38,8.120000,2.910000
107,1,11.60,12.180000,-1.460000
108,5,14.10,5.670000,-0.230000


In [16]:
# creating windowed dataset
# data windowing function test case passed
def create_windowed_dataset(series, window_size,shift_size, batch_size,shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size , shift = shift_size, drop_remainder = True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:,1:],window[window_size - 1,0]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset
df_windowed = create_windowed_dataset(df, 20, 10, 1, 128)

In [34]:
# codding lstm layers and cnn
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: x,
                      input_shape=[20, 3]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  tf.keras.layers.Conv2D(64,5,strides=(2,2)),
  tf.keras.layers.MaxPooling2D((2, 2), strides = (2,2)),
  
  tf.keras.layers.Conv2D(128,3,strides=(1,1)),  
  # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  tf.keras.layers.Dense(6, activation='softmax')
  
])

In [35]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_15 (Lambda)           (None, 20, 3)             0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 20, 64)            9216      
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 20, 64)            24832     
_________________________________________________________________
lambda_16 (Lambda)           (None, 20, 64, 1)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 30, 64)         1664      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 15, 64)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 2, 13, 128)      

In [37]:
optimizer = tf.keras.optimizers.SGD(lr=8e-6, momentum=0.9)
model.compile(loss="mse", optimizer=optimizer)

history = model.fit(df_windowed, epochs=100, verbose=1)

Epoch 1/100
10/10 [==============================] - 7s 20ms/step - loss: 11.8450
Epoch 2/100
10/10 [==============================] - 0s 19ms/step - loss: 11.7229
Epoch 3/100
10/10 [==============================] - 0s 18ms/step - loss: 12.1633
Epoch 4/100
10/10 [==============================] - 0s 19ms/step - loss: 7.6895
Epoch 5/100
10/10 [==============================] - 0s 19ms/step - loss: 12.8229
Epoch 6/100
10/10 [==============================] - 0s 18ms/step - loss: 13.1264
Epoch 7/100
10/10 [==============================] - 0s 20ms/step - loss: 10.1593
Epoch 8/100
10/10 [==============================] - 0s 19ms/step - loss: 6.0743
Epoch 9/100
10/10 [==============================] - 0s 19ms/step - loss: 8.9264
Epoch 10/100
10/10 [==============================] - 0s 21ms/step - loss: 9.8243
Epoch 11/100
10/10 [==============================] - 0s 20ms/step - loss: 12.4633
Epoch 12/100
10/10 [==============================] - 0s 18ms/step - loss: 9.9007
Epoch 13/100
10/10